## ChatGPT Investment Advice

#### This script automates investment advice using the ChatGPT API.

Using the Python script together with the ChatGPT API allows this chatbot to be used on an commercial scale. I.e., it fully automates investment advice and can easy be integrated into a webpage or other such user interface.

The chatbot replicates front-of-house investment advice that you might recieve at a commerial bank.  

In [2]:
import json
import os
from collections import OrderedDict
from datetime import datetime
from pathlib import Path

from openai import OpenAI   # pip install openai>=1.0.0
from openai_key import OPENAI_API_KEY


#### Step 1: Define the questions

In [3]:
QUESTIONS = OrderedDict({
    "objectives_time_horizon":
        ("1. Investment Objectives & Horizon:\n"
         "   What are your specific financial goals (retirement, property "
         "acquisition, education funding), and what is your anticipated timeline for "
         "accessing these funds?\n> "),

    "risk_tolerance_capacity":
        ("2. Risk Tolerance Assessment:\n"
         "   What level of portfolio volatility are you comfortable with, and what "
         "maximum temporary decline (percentage or monetary value) could you tolerate "
         "without altering your investment strategy?\n> "),

    "current_financial_position":
        ("3. Financial Position Overview:\n"
         "   Please summarize your current income, major assets, liabilities, and "
         "emergency reserves to determine your optimal investment capacity.\n> "),

    "liquidity_cash_flow_needs":
        ("4. Liquidity Requirements:\n"
         "   Do you anticipate significant expenditures (property down payment, "
         "healthcare costs, career transition) within the next 1-5 years that would "
         "necessitate liquidity from this portfolio?\n> "),

    "constraints_preferences_tax":
        ("5. Investment Parameters:\n"
         "   Please identify any relevant investment restrictions, ESG preferences,  "
         "tax considerations, or jurisdictional factors that should guide your "
         "portfolio construction.\n> "),
})

#### Step 2. Collect responses from the user

In [4]:
def ask_questions() -> OrderedDict:
    print("\n--- Investor Profile Questionnaire ---\n")
    answers = OrderedDict()
    for key, prompt in QUESTIONS.items():
        answers[key] = input(prompt).strip()
        print("\n")
    return answers

In [5]:
def save_json(data: OrderedDict, folder: str = "client_profiles") -> Path:
    Path(folder).mkdir(exist_ok=True)
    filename = Path(folder) / f"profile_{datetime.now():%Y%m%d_%H%M%S}.json"
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=2)
    return filename

In [6]:
answers = ask_questions()


--- Investor Profile Questionnaire ---













In [7]:
save_path = save_json(answers)
print(f"\nResponses saved to {save_path.resolve()}\n")


Responses saved to /Users/justincase/PycharmProjects/FIN710/Lecture7/client_profiles/profile_20251009_213644.json



#### Step 3. Build the prompt & call ChatGPT

In [8]:
SYSTEM_PROMPT = (
    "Forget all your previous instructions. Pretend you are an expert financial advisor. "
    "You are an expert financial advisor with investment and portfolio allocation experience."
)

def call_chatgpt(q_and_a: OrderedDict, model: str = "gpt-4o-mini") -> str:
    """Send the Q&A context plus the instruction to ChatGPT and return the reply."""
    # Convert Q&A to a readable block
    qa_block = "\n".join(
        f"{idx}. Q: {QUESTIONS[key].split(':')[0]}  A: {answer}"
        for idx, (key, answer) in enumerate(q_and_a.items(), start=1)
    )

    user_message = (
        f"Your client has responsed to five onboarding questions as follows:\n{qa_block}\n\n"
        "Given the situation described by the responses to the five investment questions, "
        "what investment strategy should be followed? Provide two clear and concise sentences "
        "that explain this choice of strategy without financial jargon."
        "Subsequently, identify three suitable funds."
    )

    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": user_message},
        ],
        temperature=0  # deterministic
    )

    return response.choices[0].message.content.strip()

In [9]:
client = OpenAI(api_key=OPENAI_API_KEY)

strategy = call_chatgpt(answers)

In [10]:
print(strategy)

Based on your goal to retire in 10 years and your willingness to accept some risk, a growth-oriented investment strategy is appropriate. This means focusing on investments that have the potential for higher returns, even if they come with some short-term fluctuations.

Here are three suitable funds for your investment strategy:

1. **Fisher Funds Growth Fund** - This fund invests in a diversified portfolio of growth assets, aiming for long-term capital appreciation.
2. **Smartshares NZ Top 50 ETF** - This exchange-traded fund tracks the performance of the top 50 companies listed on the NZX, providing exposure to the New Zealand equity market.
3. **Vanguard Global Growth Fund** - This fund invests in a diversified range of global growth companies, offering potential for higher returns through international exposure.
